# Ad-hoc network model 
## Imports

In [54]:
import numpy as np
import pandas as pd 
import ipywidgets as widgets
import tqdm


from time import sleep

## Loading data

In [7]:
csv_path = widgets.Text(placeholder="Path to .csv file")
display(csv_path)

Text(value='', placeholder='Path to .csv file')

In [13]:
data = pd.read_csv(csv_path.value)
data.head()

,id,timestamp,x,y,z
0,534018,0,-37.115440,-1462.569946,-100.000000
1,544258,0,-37.027275,-1462.423096,-100.000000
2,517122,0,39.134022,-1488.308350,28.503515
3,2818,0,-11.323400,-1440.091431,31.578899
4,296962,0,-194.265503,-1672.670654,33.570766


## Preprocessing

The goal here is to filter redundant objects (for example, objects that are currently chilling in the object pool) and split data into small pieces by the timeframe 

In [42]:
filtered_data = data[data.z != -100.0] # Preserve only the objects that are currently somewhere on the map

frames = filtered_data.groupby('timestamp') # Group data into dataframes with equal timestamp, now each group is a snapshot of a moment in time during recording
group_keys = frames.groups.keys() # Now we can iterate over groups

## Model setup

In [ ]:
# TODO widget for parameter initialization
# Proposed parameters are: Connectivity distance, something about message generation frequency, ???

## Your actor implementation

In [67]:
# TODO implement actor
# Actor should be a class that allows for message sending and receiving 

## Model running

In [ ]:
# TODO run model, accumulate some statistics
# Proposed stats are: 
# 1. Connection stability (stability metric could be connection lifetime divided over the simulation length) equipped with standard stat tools (mean, stdev, median, possibly distribution, smth else)
# 2. Stats on message delivery efficiency (percentage of successful deliveries, for instance) 

## Statistics

In [ ]:
# TODO Show collected stats 